In [1]:
import cv2  # openvv 读取的格式是BGR
import matplotlib.pyplot as plt
import numpy as np
# 当调用 matplotlib.pyplot 的绘图函数 plot() 进行绘图的时候，或者生成一个 figure 画布的时候，可以直接在你的 python console 里面生成图像
%matplotlib inline  

In [2]:
# 可以定义一个方法专门用于显示图像
def cv_show(name, img):
    '''
    :param name: 窗口名字
    :param img: 显示的图像
    '''
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 图像金字塔

- 高斯金字塔
- 拉普拉斯金字塔

![title](./data/Pyramid_1.png)


## 高斯金字塔
向下采样方法（缩小）
![title](./data/Pyramid_2.png)

向上采样方法（放大）
![title](./data/Pyramid_3.png)

In [17]:
img = cv2.imread('./data/AM.png')
cv_show('img', img)
print(img.shape)

(442, 340, 3)


In [14]:
# 执行上采样，放大图像
up = cv2.pyrUp(img)
cv_show('up', up)
print(up.shape)

(884, 680, 3)


In [15]:
# 执行下采样，缩小图像
down = cv2.pyrDown(img)
cv_show('down', down)
print(down.shape)

(221, 170, 3)


In [19]:
# 先进行上采样，再进行下采样
# 会发现图像和原图相比，虽然大小一样，但是图像更模糊
up = cv2.pyrUp(img)
up_down = cv2.pyrDown(up)
cv_show('up_down', np.hstack((img, up_down)))  # 与原图进行比较

In [18]:
# 先进行下采样，再进行上采样
# 会发现图像和原图相比，虽然大小一样，但是图像更模糊
down = cv2.pyrDown(img)
down_up = cv2.pyrUp(down)
cv_show('up_down', np.hstack((img, down_up)))  # 与原图进行比较

## 拉普拉斯金字塔
就是图像减去先经过下采样再经过上采样的图
![title](./data/Pyramid_4.png)

In [20]:
down = cv2.pyrDown(img)
down_up = cv2.pyrUp(down)
lpls = img-down_up
cv_show('lpls', lpls)

# 轮廓检测

**cv2.findContours(img, mode, method)该函数会返回三个值，img(OpenCV3, CV2没有该返回值)，轮廓和每个轮廓的属性**
- mode: 轮廓检索模式
  - RETR_EXTERNAL: 只检索最外面的轮廓
  - RETR_LIST: 检索所有的轮廓，并将其保存到一条链表当中
  - RETR_CCOMP: 检索所有的轮廓，并将他们朱志伟两层：顶层是各部分的外部边界，第二层是空洞的边界
  - RETR_TREE: 检索所有的轮廓，并重构嵌套轮廓的整个层次
- method：轮廓逼近方法
  - CHAIN_APPROX_NONE：以Freeman链码的方式输出轮廓，所有其他方法输出多边形（顶点的序列），下图左图
  - CHAIN_APPROX_SIMPLE:压缩水平的、垂直的和斜的部分，也就是，函数只保留他们的终点部分，下图右图
  
  
![title](./data/chain.png)

> 为了更高的准确率，使用二值图像

In [21]:
img = cv2.imread('./data/contours.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
cv_show('thresh', thresh)

In [22]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,
                                              cv2.CHAIN_APPROX_NONE)

In [25]:
# 绘制轮廓
# 传入绘制图像，轮廓，轮廓索引，颜色模式，线条厚度
# 注意需要 copy ，否则原图会发生改变

draw_img = img.copy()
res = cv2.drawContours(draw_img, contours, -1, (0, 0, 255), 2)
cv_show('res', res)

In [26]:
# 轮廓特征
cnt = contours[0]
# 面积
cv2.contourArea(cnt)

8500.5

In [30]:
# 周长，True表示闭合的
cv2.arcLength(cnt, True)

437.9482651948929

# 轮廓近似
![title](./data/contours3.png)

In [27]:
img = cv2.imread('./data/contours2.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,
                                      cv2.CHAIN_APPROX_NONE)
cnt = contours[0]

draw_img = img.copy()
res = cv2.drawContours(draw_img, [cnt], -1, (0, 0, 255), 2)
cv_show('res', res)

In [28]:
epsilon = 0.15*cv2.arcLength(cnt, True)
approx = cv2.approxPolyDP(cnt, epsilon, True)

draw_img = img.copy()
res = cv2.drawContours(draw_img, [approx], -1, (0, 0, 255), 2)
cv_show("res", res)

In [29]:
# 边界矩形
img = cv2.imread('./data/contours.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnt = contours[0]

x,y,w,h = cv2.boundingRect(cnt)
img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
cv_show("img", img)

In [39]:
area = cv2.contourArea(cnt)
x, y, w, h = cv2.boundingRect(cnt)
rect_area = w * h
extent = float(area) / rect_area
print ('轮廓面积与边界矩形比',extent)

轮廓面积与边界矩形比 0.5154317244724715


In [43]:
# 外接圆
img = cv2.imread('./data/contours.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnt = contours[0]

(x,y),radius = cv2.minEnclosingCircle(cnt) 
center = (int(x),int(y)) 
radius = int(radius) 
img = cv2.circle(img,center,radius,(0,255,0),2)
cv_show("img", img)